Build a Convolutional Neural Network, like what we built in lectures to classify the images across all 10 classes in CIFAR 10. You need to adjust the fully connected layer at the end properly concerning the number of output classes. Train your network for 200 epochs

In [1]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
import torch
import time

# Define the data path and Load CIFAR-10 dataset
datapath = '../data-unversions/p1ch7/'
cifar10 = datasets.CIFAR10(root= datapath, train=True, download = True, transform=transforms.ToTensor())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

100%|██████████| 170M/170M [00:13<00:00, 12.5MB/s]


Extracting ../data-unversions/p1ch7/cifar-10-python.tar.gz to ../data-unversions/p1ch7/


Pre-processing. Seperating training and validation datasets.

In [2]:
# Compute dataset mean and standard deviation for normalization
imgs = torch.stack([img_t for img_t, _ in cifar10], dim=3)
mean = imgs.view(3, -1).mean(dim=1)
std = imgs.view(3, -1).std(dim=1)

# Define normalization transformation
normalize_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=dataset_mean, std=dataset_std)
])

train_data = datasets.CIFAR10(root=data_path, train=True, download=True, transform=normalize_transform)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

val_data = datasets.CIFAR10(root=data_path, train=False, download=True, transform=normalize_transform)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


Convolution Neural Network Training and Validation

In [4]:
class CIFAR10Net(nn.Module):
    def __init__(self):
        super(CIFAR10Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.tanh = nn.Tanh()
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.flatten(x)
        x = self.tanh(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model
model = CIFAR10Net().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

epochs = 200
total_start_time = time.time()
for epoch in range(epochs):
    start_time = time.time()
    model.train()
    losses = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        losses += loss.item()

    end_time = time.time()
    training_time = end_time - start_time

    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {losses/len(train_loader)}, Training Time: {training_time:.2f} seconds')

total_end_time = time.time()
total_training_time = total_end_time - total_start_time
print(f'Total Training Time: {total_training_time:.2f} seconds')

model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()\

val_accuracy = correct / total
print(f'Test Accuracy: {val_accuracy * 100:.2f}%')

Epoch 1/200, Loss: 2.1598667445582453, Training Time: 16.75 seconds
Epoch 11/200, Loss: 1.2879153241427832, Training Time: 16.33 seconds
Epoch 21/200, Loss: 1.0669298431313503, Training Time: 16.87 seconds
Epoch 31/200, Loss: 0.8988889748479644, Training Time: 16.53 seconds
Epoch 41/200, Loss: 0.752425080945838, Training Time: 16.14 seconds
Epoch 51/200, Loss: 0.6081888225413407, Training Time: 16.72 seconds
Epoch 61/200, Loss: 0.464308940593966, Training Time: 16.64 seconds
Epoch 71/200, Loss: 0.32798325690373503, Training Time: 16.12 seconds
Epoch 81/200, Loss: 0.2137395437527984, Training Time: 16.57 seconds
Epoch 91/200, Loss: 0.1338190478764317, Training Time: 16.57 seconds
Epoch 101/200, Loss: 0.08530234694433228, Training Time: 16.50 seconds
Epoch 111/200, Loss: 0.05731386684062423, Training Time: 16.18 seconds
Epoch 121/200, Loss: 0.04106898987767068, Training Time: 15.93 seconds
Epoch 131/200, Loss: 0.031129694405540358, Training Time: 15.96 seconds
Epoch 141/200, Loss: 0.0246

 Extend your CNN by adding one more additional convolution layer followed by an activation function and pooling function. You also need to adjust your fully connected layer properly with respect to intermediate feature dimensions. Train your network for 200 epochs.

In [5]:
class ExtendedCIFAR10Net(nn.Module):
    def __init__(self):
        super(ExtendedCIFAR10Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 4 * 4, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.maxpool(x)
        x = self.relu(self.conv2(x))
        x = self.maxpool(x)
        x = self.relu(self.conv3(x))
        x = self.maxpool(x)
        x = x.view(-1, 128 * 4 * 4)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# Instantiate the model
model = ExtendedCIFAR10Net().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Training loop
epochs = 200
total_start_time = time.time()
for epoch in range(epochs):
    start_time = time.time()
    model.train()
    losses = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        losses += loss.item()

    end_time = time.time()
    training_time = end_time - start_time

    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {losses/len(train_loader)}, Training Time: {training_time:.2f} seconds')


total_end_time = time.time()
total_training_time = total_end_time - total_start_time
print(f'Total Training Time: {total_training_time:.2f} seconds')

model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

val_accuracy = correct / total
print(f'Test Accuracy: {val_accuracy * 100:.2f}%')

Epoch 1/200, Loss: 2.2927755951042443, Training Time: 17.25 seconds
Epoch 11/200, Loss: 1.4788093929937538, Training Time: 17.81 seconds
Epoch 21/200, Loss: 1.202768693462977, Training Time: 17.60 seconds
Epoch 31/200, Loss: 1.0253927582590074, Training Time: 17.11 seconds
Epoch 41/200, Loss: 0.8864922707308124, Training Time: 16.78 seconds
Epoch 51/200, Loss: 0.7688095677913341, Training Time: 16.85 seconds
Epoch 61/200, Loss: 0.6643369033103255, Training Time: 17.35 seconds
Epoch 71/200, Loss: 0.5681727902578835, Training Time: 17.48 seconds
Epoch 81/200, Loss: 0.4747468213755125, Training Time: 17.07 seconds
Epoch 91/200, Loss: 0.38337138096396156, Training Time: 16.92 seconds
Epoch 101/200, Loss: 0.2950749139296116, Training Time: 17.33 seconds
Epoch 111/200, Loss: 0.2111034370834867, Training Time: 16.74 seconds
Epoch 121/200, Loss: 0.13933392756774077, Training Time: 17.60 seconds
Epoch 131/200, Loss: 0.08268350430645682, Training Time: 17.50 seconds
Epoch 141/200, Loss: 0.038976